# Sheath Expansion Simulations Results - Analysis Round 3
Third from a series of notebooks analysing the sheath expansion simulations in a manner similar to Bergmann in his 2002 paper. This one analyses the previously combined xarray dataset (sheath_exp_fwp_ivs.nc) which contains the most up to date (as of 2020-12-11) simulations for the the 2D probes. This is non-piecewise!

In [50]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 14
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu
import flopter.spice.xrfuncs as xrf

In [51]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [52]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [53]:
skippable_scans = set() 
single_sims = set()
non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

In [54]:
# scans_searchstr = '*/*/sheath_exp/*'
# scans_searchstr = '*/*/sheath_exp_fwp/*'
scans_searchstr = [
    '*/*/sheath_exp_hg/*', 
    '*/*/sheath_exp_fwp/*',
]

scans, all_run_dirs = xrf.get_run_dirs(scans_searchstr)

[0]: marconi/spice2/sheath_exp_fwp/angled_flush_as
	[0,0]: angled_flush_as/alpha_yz_-10.0
	[0,1]: angled_flush_as/alpha_yz_-12.0
	[0,2]: angled_flush_as/alpha_yz_-2.0
	[0,3]: angled_flush_as/alpha_yz_-20.0
	[0,4]: angled_flush_as/alpha_yz_-3.0
	[0,5]: angled_flush_as/alpha_yz_-30.0
	[0,6]: angled_flush_as/alpha_yz_-4.0
	[0,7]: angled_flush_as/alpha_yz_-5.0
	[0,8]: angled_flush_as/alpha_yz_-6.0
	[0,9]: angled_flush_as/alpha_yz_-8.0
[1]: marconi/spice2/sheath_exp_fwp/angled_recessed_as
	[1,0]: angled_recessed_as/alpha_yz_-10.0
	[1,1]: angled_recessed_as/alpha_yz_-12.0
	[1,2]: angled_recessed_as/alpha_yz_-2.0
	[1,3]: angled_recessed_as/alpha_yz_-20.0
	[1,4]: angled_recessed_as/alpha_yz_-3.0
	[1,5]: angled_recessed_as/alpha_yz_-30.0
	[1,6]: angled_recessed_as/alpha_yz_-4.0
	[1,7]: angled_recessed_as/alpha_yz_-5.0
	[1,8]: angled_recessed_as/alpha_yz_-6.0
	[1,9]: angled_recessed_as/alpha_yz_-8.0
[2]: marconi/spice2/sheath_exp_fwp/flat_10
	[2,0]: flat_10/alpha_yz_-10.0
[3]: marconi/spice2/she

In [55]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/sheath_exp_fwp/angled_flush_as
	[0,0]: angled_flush_as/alpha_yz_-10.0
	[0,1]: angled_flush_as/alpha_yz_-12.0
	[0,2]: angled_flush_as/alpha_yz_-2.0
	[0,3]: angled_flush_as/alpha_yz_-20.0
	[0,4]: angled_flush_as/alpha_yz_-3.0
	[0,5]: angled_flush_as/alpha_yz_-30.0
	[0,6]: angled_flush_as/alpha_yz_-4.0
	[0,7]: angled_flush_as/alpha_yz_-5.0
	[0,8]: angled_flush_as/alpha_yz_-6.0
	[0,9]: angled_flush_as/alpha_yz_-8.0
[1]: marconi/spice2/sheath_exp_fwp/angled_recessed_as
	[1,0]: angled_recessed_as/alpha_yz_-10.0
	[1,1]: angled_recessed_as/alpha_yz_-12.0
	[1,2]: angled_recessed_as/alpha_yz_-2.0
	[1,3]: angled_recessed_as/alpha_yz_-20.0
	[1,4]: angled_recessed_as/alpha_yz_-3.0
	[1,5]: angled_recessed_as/alpha_yz_-30.0
	[1,6]: angled_recessed_as/alpha_yz_-4.0
	[1,7]: angled_recessed_as/alpha_yz_-5.0
	[1,8]: angled_recessed_as/alpha_yz_-6.0
	[1,9]: angled_recessed_as/alpha_yz_-8.0
[2]: marconi/spice2/sheath_exp_fwp/flat_10
	[2,0]: flat_10/alpha_yz_-10.0
[3]: marconi/spice2/she

In [56]:
spl_path = spice_dir / all_run_dirs[scans[12]][1]
print(spl_path)
spl_backup_path = list(spl_path.glob('backup_*'))[-1]

spl_coarse_path = spice_dir / all_run_dirs[scans[5]][1]
print(spl_coarse_path)
spl_coarse_bu_path = list(spl_coarse_path.glob('backup_*'))[-1]

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-12.0
/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_flush_as/alpha_yz_-12.0


In [57]:
# probe_desig = {
#     'angled_flush': {'theta_p':10.0, 'recession':0.0},
#     'angled_semi-recessed': {'theta_p':10.0, 'recession':5.0e-4},
#     'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3},
#     'semi-angled_flush': {'theta_p':5.0, 'recession':0.0},
#     'semi-angled_semi-recessed': {'theta_p':5.0, 'recession':5.0e-4},
#     'semi-angled_recessed': {'theta_p':5.0, 'recession':1.0e-3},
#     'flat_flush': {'theta_p':0.0, 'recession':0.0},
#     'flat_semi-recessed': {'theta_p':0.0, 'recession':5.0e-4},
#     'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3},
# }

# cb_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
#               '#f781bf', '#a65628', '#984ea3', 
#               '#999999', '#e41a1c', '#dede00']

probe_desig = {
    'angled_flush': {'theta_p':10.0, 'recession':0.0, 'gap': 1.0e-3},
    'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush': {'theta_p':0.0, 'recession':0.0, 'gap': 1.0e-3},
    'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3, 'gap': 1.0e-3},
    'flat_flush_gapless': {'theta_p':0.0, 'recession': 0.0, 'gap': 0.0}
}

cb_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00']


probe_colour = {probe_label: cb_palette[i] for i, probe_label in enumerate(probe_desig.keys())}
probe_colour

{'angled_flush': '#377eb8',
 'angled_recessed': '#ff7f00',
 'flat_flush': '#4daf4a',
 'flat_recessed': '#f781bf',
 'flat_flush_gapless': '#a65628'}

In [58]:
importlib.reload(spl)

<module 'flopter.spice.splopter' from '/home/jleland/coding/projects/flopter/flopter/spice/splopter.py'>

In [59]:
splopter =  spl.Splopter(spl_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True)
splopter_coarse =  spl.Splopter(spl_coarse_bu_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                                store_dataframe_fl=True)

parser = inp.InputParser(input_filename=spl_backup_path / 'input.inp')
denormaliser = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser) #, dimensions=3)

parser_coarse = inp.InputParser(input_filename=spl_coarse_bu_path / 'input.inp')
denormaliser_coarse = nrm.Denormaliser(dt=splopter_coarse.tdata.dt, input_parser=parser_coarse)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-12.0/backup_20201221-0640) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_flush_as/alpha_yz_-12.0/backup_20201126-0757) doesn't seem to be valid.
Continuing anyway.
dx = 1.0, dy = 0.5, dz = 0.5
dx = 1.0, dy = 1.0, dz = 1.0


In [60]:
dV = parser.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser.getfloat('mks', 'mks_te')
n_0 = parser.getfloat('mks', 'mks_n0')

L = parser.getfloat('rectangle2', 'yhigh') - parser.getfloat('rectangle2', 'ylow')
g = parser.getfloat('rectangle2', 'ylow') - parser.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
theta_p = np.radians(10.0)

print(f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {theta_p} \n")

L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 



In [61]:
print(denormaliser(1, 'current'))
print(denormaliser_coarse(1, 'current'))

print(denormaliser.K, denormaliser.N_pdc, denormaliser.dt)
print(denormaliser_coarse.K, denormaliser_coarse.N_pdc, denormaliser_coarse.dt)

0.09990109679420577
0.19980219358841153
1381587.3527102151 200.0 0.00016980762735243054
5526349.410840861 50.0 0.0003396152547048611


In [62]:
# combined_ds = xr.load_dataset('sheath_exp_ivs.nc')
# combined_ds = xr.load_dataset('new_sheath_exp_ivs.nc')
old_combined_ds = xr.load_dataset('sheath_exp_fwp_ivs.nc')
combined_ds = xr.load_dataset('sheath_exp_hg_ivs.nc').expand_dims(dim=['probe']).assign_coords(probe=['flat_flush']).assign(
    {'recession': 0.0, 'theta_p':0.0, 'theta_p_rads':0.0})

combined_ds

<xarray.Dataset>
Dimensions:               (probe: 1, theta: 15, voltage: 1201)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 ... 1.665e-15 0.025
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.3491 0.5236
  * probe                 (probe) <U10 'flat_flush'
Data variables:
    str_iv_chi2           (probe, theta) float64 723.8 802.2 ... 1.067e+03
    str_iv_reduced_chi2   (probe, theta) float64 0.6514 0.7195 ... 0.9598 0.9566
    str_iv_I_sat          (probe, theta) float64 -14.44 -21.82 ... -108.4 -150.9
    str_iv_d_I_sat        (probe, theta) float64 0.01673 0.02617 ... 0.2316
    str_iv_a              (probe, theta) float64 0.005932 0.005801 ... 0.008279
    str_iv_d_a            (probe, theta) float64 0.0001485 ... 0.0001997
    str_iv_T_e            (probe, theta) float64 1.113 1.099 ... 1.082 1.076
    str_iv_d_T_e          (probe, theta) float64 0.005743 0.005632 ... 0.00541
    str_iv_V_f            (probe, theta) float64 -2.368 -2.338 ... -2.666 -2.736
    str_iv_d_V_f          (probe, theta) float64 0.003084 0.002892 ... 0.002625
    str_iv_run_dir        (probe, theta) object 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (probe, theta) float64 -29.98 -29.98 ... -29.98 -29.98
    str_iv_voltage_max    (probe, theta) float64 -2.125 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (probe, theta) float64 1.106e+03 ... 1.176e+03
    norm_iv_reduced_chi2  (probe, theta) float64 1.003 0.9827 ... 1.051 1.081
    norm_iv_I_sat         (probe, theta) float64 -14.2 -21.51 ... -107.3 -149.3
    norm_iv_d_I_sat       (probe, theta) float64 0.01626 0.02425 ... 0.2018
    norm_iv_a             (probe, theta) float64 0.007396 0.007055 ... 0.009172
    norm_iv_d_a           (probe, theta) float64 0.0001542 ... 0.0001874
    norm_iv_voltage_min   (probe, theta) float64 -27.61 -27.64 ... -27.29 -27.25
    norm_iv_voltage_max   (probe, theta) float64 -0.01163 -0.0166 ... -0.0208
    ion_chi2              (probe, theta) float64 528.0 593.0 ... 820.2 776.8
    ion_reduced_chi2      (probe, theta) float64 0.6118 0.6864 ... 0.9649 0.916
    ion_I_sat             (probe, theta) float64 14.46 21.91 ... 108.5 150.8
    ion_d_I_sat           (probe, theta) float64 0.01913 0.03042 ... 0.2797
    ion_a                 (probe, theta) float64 0.005252 0.004938 ... 0.007918
    ion_d_a               (probe, theta) float64 0.0001623 ... 0.0002357
    ion_voltage_min       (probe, theta) float64 3.839 3.842 ... 3.843 3.844
    ion_voltage_max       (probe, theta) float64 12.05 12.06 ... 11.94 11.93
    current               (probe, theta, voltage) float64 -15.88 -15.98 ... nan
    current_e             (probe, theta, voltage) float64 0.0 0.0 ... 0.0 nan
    current_i             (probe, theta, voltage) float64 15.88 15.98 ... nan
    d_current             (probe, theta, voltage) float64 0.1944 0.2147 ... nan
    d_current_e           (probe, theta, voltage) float64 0.0 0.0 ... 0.0 nan
    d_current_i           (probe, theta, voltage) float64 0.1944 0.2147 ... nan
    time                  (probe, theta, voltage) float64 0.0 0.06436 ... nan
    recession             float64 0.0

In [63]:
halfgrid_ar_ds = xr.load_dataset('sheath_exp_hg_ar_ivs.nc').expand_dims(
    dim=['probe']
).assign_coords(probe=['angled_recessed']).assign({'recession': 3.0e-4})

halfgrid_ar_ds

<xarray.Dataset>
Dimensions:               (probe: 1, theta: 12, voltage: 1201)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 ... 1.665e-15 0.025
  * theta                 (theta) float64 4.0 5.0 6.0 7.0 ... 15.0 20.0 30.0
    v_f                   (theta) float64 -2.59 -2.58 ... -2.691 1.665e-15
  * probe                 (probe) <U15 'angled_recessed'
Data variables:
    str_iv_chi2           (probe, theta) float64 836.1 764.7 ... 1.059e+03
    str_iv_reduced_chi2   (probe, theta) float64 0.7498 0.6858 ... 0.9284 0.9509
    str_iv_I_sat          (probe, theta) float64 -57.05 -62.04 ... -132.0 -173.5
    str_iv_d_I_sat        (probe, theta) float64 0.05001 0.05535 ... 0.2467
    str_iv_a              (probe, theta) float64 0.007439 0.006849 ... 0.006466
    str_iv_d_a            (probe, theta) float64 0.0001149 ... 0.0001817
    str_iv_T_e            (probe, theta) float64 1.119 1.119 ... 1.066 1.051
    str_iv_d_T_e          (probe, theta) float64 0.003496 0.003565 ... 0.005047
    str_iv_V_f            (probe, theta) float64 -2.568 -2.582 ... -2.703 -2.709
    str_iv_d_V_f          (probe, theta) float64 0.001641 0.001682 ... 0.002491
    str_iv_run_dir        (probe, theta) object 'marconi/spice2/sheath_exp_hg/angled_recessed_4/alpha_yz_-4.0' ... 'marconi/spice2/sheath_exp_hg/angled_recessed_as/alpha_yz_-30.0'
    str_iv_voltage_min    (probe, theta) float64 -29.98 -29.98 ... -29.98 -29.98
    str_iv_voltage_max    (probe, theta) float64 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (probe, theta) float64 1.399e+03 ... 1.072e+03
    norm_iv_reduced_chi2  (probe, theta) float64 1.279 1.329 ... 1.068 0.985
    norm_iv_I_sat         (probe, theta) float64 -56.21 -61.0 ... -130.6 -172.4
    norm_iv_d_I_sat       (probe, theta) float64 0.05219 0.06164 ... 0.2078
    norm_iv_a             (probe, theta) float64 0.008609 0.008307 ... 0.00697
    norm_iv_d_a           (probe, theta) float64 0.0001268 ... 0.000164
    norm_iv_voltage_min   (probe, theta) float64 -27.39 -27.39 ... -27.28 -27.26
    norm_iv_voltage_max   (probe, theta) float64 -0.01032 -0.01954 ... -0.008294
    ion_chi2              (probe, theta) float64 535.6 539.2 ... 795.1 783.1
    ion_reduced_chi2      (probe, theta) float64 0.6272 0.6314 ... 0.9354 0.9235
    ion_I_sat             (probe, theta) float64 57.36 62.31 ... 132.1 173.4
    ion_d_I_sat           (probe, theta) float64 0.05501 0.06384 ... 0.3033
    ion_a                 (probe, theta) float64 0.006214 0.00579 ... 0.006237
    ion_d_a               (probe, theta) float64 0.0001193 ... 0.000219
    ion_voltage_min       (probe, theta) float64 3.839 3.843 ... 3.838 3.838
    ion_voltage_max       (probe, theta) float64 11.97 11.97 ... 11.94 11.93
    voltage_wall          (probe, theta, voltage) float64 0.0 0.0 ... 0.0 nan
    current               (probe, theta, voltage) float64 -62.37 -62.06 ... nan
    current_e             (probe, theta, voltage) float64 0.0 0.0 ... 0.0 nan
    current_i             (probe, theta, voltage) float64 62.37 62.06 ... nan
    d_current             (probe, theta, voltage) float64 0.5575 0.5733 ... nan
    d_current_e           (probe, theta, voltage) float64 0.0 0.0 ... 0.0 nan
    d_current_i           (probe, theta, voltage) float64 0.5575 0.5733 ... nan
    time                  (probe, theta, voltage) float64 0.0 0.03226 ... nan
    voltage_corr          (probe, theta, voltage) float64 -27.39 -27.36 ... nan
    voltage_cl            (probe, theta, voltage) float64 11.97 11.96 ... nan
    recession             float64 0.0003

In [64]:
tempscan_ds = xr.load_dataset('se_tempscan_ivs.nc')
tempscan_ds

<xarray.Dataset>
Dimensions:               (temperature: 4, theta: 6, voltage: 1777)
Coordinates:
  * temperature           (temperature) float64 5.0 10.0 20.0 50.0
  * theta                 (theta) float64 2.0 3.0 5.0 8.0 10.0 30.0
  * voltage               (voltage) float64 -29.97 -29.95 -29.93 ... 5.0 5.025
    probe                 (temperature) object 'flat_flush_as' ... 'flat_flush_T50'
    recession             (temperature) float64 0.0 0.0 0.0 0.0
    recession_desc        (temperature) object 'flush' 'flush' 'flush' 'flush'
    recessed_fl           (temperature) bool False False False False
    theta_p               (temperature) float64 0.0 0.0 0.0 0.0
    theta_p_desc          (temperature) object 'flat' 'flat' 'flat' 'flat'
    angled_fl             (temperature) bool False False False False
    theta_p_rads          (temperature) float64 0.0 0.0 0.0 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.1745 0.5236
Data variables:
    str_iv_chi2           (temperature, theta) float64 4.549e+03 ... 637.6
    str_iv_reduced_chi2   (temperature, theta) float64 4.08 5.962 ... 6.372 1.24
    str_iv_I_sat          (temperature, theta) float64 -7.115 -10.62 ... -30.73
    str_iv_d_I_sat        (temperature, theta) float64 0.04334 ... 0.2516
    str_iv_a              (temperature, theta) float64 0.006474 ... 0.01245
    str_iv_d_a            (temperature, theta) float64 0.00111 ... 0.0009897
    str_iv_T_e            (temperature, theta) float64 1.538 1.429 ... 0.4902
    str_iv_d_T_e          (temperature, theta) float64 0.01072 ... 0.03979
    str_iv_V_f            (temperature, theta) float64 -4.628 -4.493 ... -1.707
    str_iv_d_V_f          (temperature, theta) float64 0.009595 ... 0.04259
    str_iv_run_dir        (temperature, theta) object 'marconi/spice2/sheath_exp_fwp/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/se_tempscan/flat_flush_T50/alpha_yz_-30.0'
    str_iv_voltage_min    (temperature, theta) float64 -29.97 -29.97 ... -14.97
    str_iv_voltage_max    (temperature, theta) float64 -2.025 -2.025 ... -2.025
    norm_iv_chi2          (temperature, theta) float64 695.7 780.1 ... 9.895e+05
    norm_iv_reduced_chi2  (temperature, theta) float64 0.6861 ... 1.243e+03
    norm_iv_I_sat         (temperature, theta) float64 -6.516 -9.839 ... -2.305
    norm_iv_d_I_sat       (temperature, theta) float64 0.01378 0.0216 ... 8.924
    norm_iv_a             (temperature, theta) float64 0.01583 0.01325 ... 1.5
    norm_iv_d_a           (temperature, theta) float64 0.0003211 ... 6.354
    norm_iv_voltage_min   (temperature, theta) float64 -25.38 -25.57 ... -19.97
    norm_iv_voltage_max   (temperature, theta) float64 -0.0008451 ... -0.025
    ion_chi2              (temperature, theta) float64 513.2 556.5 ... 1.411e+03
    ion_reduced_chi2      (temperature, theta) float64 0.601 0.6463 ... 2.219
    ion_I_sat             (temperature, theta) float64 6.573 9.791 ... 22.86
    ion_d_I_sat           (temperature, theta) float64 0.01636 ... 0.2895
    ion_a                 (temperature, theta) float64 0.01465 ... 0.06393
    ion_d_a               (temperature, theta) float64 0.0003592 ... 0.002738
    ion_voltage_min       (temperature, theta) float64 2.829 2.841 ... 2.842
    ion_voltage_max       (temperature, theta) float64 11.31 11.37 ... 9.449
    current               (temperature, theta, voltage) float64 -7.778 ... -0.0
    current_e             (temperature, theta, voltage) float64 0.0 0.0 ... 0.0
    current_i             (temperature, theta, voltage) float64 7.778 ... 0.0
    d_current             (temperature, theta, voltage) float64 0.2109 ... 0.0
    d_current_e           (temperature, theta, voltage) float64 0.0 0.0 ... 0.0
    d_current_i           (temperature, theta, voltage) float64 0.2109 ... 0.0
    time                  (temperature, theta, voltage) float64 0.0 ... 5.038

In [65]:
piecewise_ds = xr.load_dataset('piecewise_sheath_exp_ivs.nc')
piecewise_ds

<xarray.Dataset>
Dimensions:         (probe: 1, theta: 7, voltage: 14, y: 557, z: 233)
Coordinates:
  * probe           (probe) object 'flat_flush'
  * z               (z) int64 0 1 2 3 4 5 6 7 ... 226 227 228 229 230 231 232
  * voltage         (voltage) float64 -25.0 -18.0 -12.0 -10.0 ... -2.0 -1.0 0.0
  * y               (y) int64 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * theta           (theta) float64 3.0 5.0 15.0 20.0 12.0 10.0 8.0
Data variables:
    current         (probe, theta, voltage) float64 10.47 10.23 ... -184.3
    d_current       (probe, theta, voltage) float64 3.618 3.592 ... 13.93 16.11
    derr_current    (probe, theta, voltage) float64 0.007561 ... 0.03366
    current_i       (probe, theta, voltage) float64 10.47 10.23 ... 14.75 10.22
    d_current_i     (probe, theta, voltage) float64 3.618 3.592 ... 3.924 3.235
    derr_current_i  (probe, theta, voltage) float64 0.007561 ... 0.006761
    current_e       (probe, theta, voltage) float64 -3.1e-06 ... -194.6
    d_current_e     (probe, theta, voltage) float64 0.001761 0.003295 ... 15.97
    derr_current_e  (probe, theta, voltage) float64 3.68e-06 ... 0.03337
    rho             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 111.5
    pot             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 0.0 0.0
    delta           (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    delta_w         (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    theta_rads      (probe, theta) float64 -0.05236 -0.08727 ... -0.1745 -0.1396
    theta_p_rads    (probe) object 'flat_flush'
    recession       (probe) object 'flat_flush'

In [66]:
floating_ds = xr.load_dataset('sheath_exp_fflwp_ivs.nc').expand_dims(dim=['probe']).assign_coords(probe=['flat_flush']).assign(
    {'recession': 0.0, 'theta_p':0.0, 'theta_p_rads':0.0})
floating_ds

<xarray.Dataset>
Dimensions:               (probe: 1, theta: 11, voltage: 1204)
Coordinates:
  * voltage               (voltage) float64 -29.97 -29.95 -29.93 ... 0.075 0.1
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.3491 0.5236
  * probe                 (probe) <U10 'flat_flush'
Data variables:
    str_iv_chi2           (probe, theta) float64 1.266e+04 ... 1.269e+03
    str_iv_reduced_chi2   (probe, theta) float64 11.36 4.923 ... 1.181 1.139
    str_iv_I_sat          (probe, theta) float64 -7.46 -10.24 ... -43.8 -70.93
    str_iv_d_I_sat        (probe, theta) float64 0.09394 0.07669 ... 0.2359
    str_iv_a              (probe, theta) float64 9.645e-16 0.01289 ... 0.009413
    str_iv_d_a            (probe, theta) float64 0.002817 0.001507 ... 0.0004422
    str_iv_T_e            (probe, theta) float64 2.2 1.773 ... 0.9872 1.096
    str_iv_d_T_e          (probe, theta) float64 0.03114 0.01516 ... 0.01094
    str_iv_V_f            (probe, theta) float64 -4.755 -4.613 ... -3.244 -2.863
    str_iv_d_V_f          (probe, theta) float64 0.01731 0.01063 ... 0.005426
    str_iv_run_dir        (probe, theta) object 'marconi/spice2/sheath_exp_fflwp/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_fflwp/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (probe, theta) float64 -29.97 -29.97 ... -29.97 -29.97
    str_iv_voltage_max    (probe, theta) float64 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (probe, theta) float64 1.624e+03 ... 1.298e+03
    norm_iv_reduced_chi2  (probe, theta) float64 1.611 1.593 ... 1.062 1.199
    norm_iv_I_sat         (probe, theta) float64 -6.221 -8.865 ... -44.02 -69.9
    norm_iv_d_I_sat       (probe, theta) float64 0.02073 0.03008 ... 0.1993
    norm_iv_a             (probe, theta) float64 0.02235 0.02772 ... 0.01057
    norm_iv_d_a           (probe, theta) float64 0.00053 0.0005568 ... 0.0003992
    norm_iv_voltage_min   (probe, theta) float64 -25.23 -25.44 ... -26.7 -27.13
    norm_iv_voltage_max   (probe, theta) float64 -0.001174 ... -0.001578
    ion_chi2              (probe, theta) float64 462.0 553.0 ... 826.9 943.2
    ion_reduced_chi2      (probe, theta) float64 0.6016 0.7127 ... 1.002 1.119
    ion_I_sat             (probe, theta) float64 6.622 9.67 ... 44.39 70.89
    ion_d_I_sat           (probe, theta) float64 0.02051 0.03282 ... 0.2913
    ion_a                 (probe, theta) float64 0.0138 0.01568 ... 0.008845
    ion_d_a               (probe, theta) float64 0.0004289 ... 0.0005266
    ion_voltage_min       (probe, theta) float64 3.834 3.842 ... 3.844 3.834
    ion_voltage_max       (probe, theta) float64 11.26 11.33 ... 11.74 11.89
    current               (probe, theta, voltage) float64 -7.915 ... 738.7
    current_e             (probe, theta, voltage) float64 0.0 0.0 ... -806.3
    current_i             (probe, theta, voltage) float64 7.915 7.788 ... 67.62
    d_current             (probe, theta, voltage) float64 0.2041 0.206 ... 8.866
    d_current_e           (probe, theta, voltage) float64 0.0 0.0 ... 9.399
    d_current_i           (probe, theta, voltage) float64 0.2041 0.206 ... 2.089
    time                  (probe, theta, voltage) float64 0.0 0.06419 ... 5.307
    theta_p               float64 0.0
    recession             float64 0.0

In [67]:
fwp2_ds = xr.load_dataset('sheath_exp_fwp-2_ivs.nc').expand_dims(
    dim=['probe']
).assign_coords(
    probe=['flat_flush']
).assign({
    'recession': 0.0, 
    'theta_p':0.0, 
    'theta_p_rads':0.0
})
fwp2_ds

<xarray.Dataset>
Dimensions:               (probe: 1, theta: 11, voltage: 1204)
Coordinates:
  * voltage               (voltage) float64 -29.97 -29.95 -29.93 ... 0.075 0.1
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.3491 0.5236
  * probe                 (probe) <U10 'flat_flush'
Data variables:
    str_iv_chi2           (probe, theta) float64 488.0 4.618e+03 ... 1.197e+03
    str_iv_reduced_chi2   (probe, theta) float64 0.5824 4.142 ... 1.392 1.074
    str_iv_I_sat          (probe, theta) float64 -6.478 -10.55 ... -44.14 -70.85
    str_iv_d_I_sat        (probe, theta) float64 0.1061 0.06414 ... 0.2297
    str_iv_a              (probe, theta) float64 0.0297 0.006142 ... 0.01018
    str_iv_d_a            (probe, theta) float64 0.01001 0.001074 ... 0.0004292
    str_iv_T_e            (probe, theta) float64 2.725 1.491 ... 0.9667 1.098
    str_iv_d_T_e          (probe, theta) float64 1.246 0.01105 ... 0.01038
    str_iv_V_f            (probe, theta) float64 0.3754 -4.454 ... -3.33 -2.854
    str_iv_d_V_f          (probe, theta) float64 3.899 0.009809 ... 0.0052
    str_iv_run_dir        (probe, theta) object 'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_fwp/flat_flush_wp-2_as_1_/alpha_yz_-30.0'
    str_iv_voltage_min    (probe, theta) float64 -29.97 -29.97 ... -29.97 -29.97
    str_iv_voltage_max    (probe, theta) float64 -8.95 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (probe, theta) float64 506.1 919.6 ... 1.175e+03
    norm_iv_reduced_chi2  (probe, theta) float64 0.6025 0.9052 ... 1.461 1.087
    norm_iv_I_sat         (probe, theta) float64 -6.366 -9.909 ... -43.61 -70.15
    norm_iv_d_I_sat       (probe, theta) float64 0.02114 0.02343 ... 0.1886
    norm_iv_a             (probe, theta) float64 0.01657 0.01229 ... 0.01072
    norm_iv_d_a           (probe, theta) float64 0.0004216 ... 0.0003763
    norm_iv_voltage_min   (probe, theta) float64 -28.97 -25.44 ... -26.78 -27.08
    norm_iv_voltage_max   (probe, theta) float64 -7.95 -0.0167 ... -0.008999
    ion_chi2              (probe, theta) float64 500.7 483.7 ... 879.8 854.9
    ion_reduced_chi2      (probe, theta) float64 0.5961 0.6234 ... 1.061 1.016
    ion_I_sat             (probe, theta) float64 6.384 9.849 ... 44.48 70.74
    ion_d_I_sat           (probe, theta) float64 0.02101 0.03081 ... 0.278
    ion_a                 (probe, theta) float64 0.01625 0.01313 ... 0.009697
    ion_d_a               (probe, theta) float64 0.0004169 ... 0.0005056
    ion_voltage_min       (probe, theta) float64 4.735 3.842 ... 3.838 3.838
    ion_voltage_max       (probe, theta) float64 12.49 11.33 ... 11.77 11.87
    voltage_wall          (probe, theta, voltage) float64 0.0 0.0 ... 0.0 0.0
    current               (probe, theta, voltage) float64 -7.857 ... 749.4
    current_e             (probe, theta, voltage) float64 0.0 0.0 ... -812.2
    current_i             (probe, theta, voltage) float64 7.857 7.984 ... 62.85
    d_current             (probe, theta, voltage) float64 0.2108 ... 9.685
    d_current_e           (probe, theta, voltage) float64 0.0 0.0 ... 9.125
    d_current_i           (probe, theta, voltage) float64 0.2108 ... 2.258
    time                  (probe, theta, voltage) float64 0.0 0.06419 ... 5.307
    theta_p               float64 0.0
    recession             float64 0.0

In [68]:
lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
fit_data_filename = 'lowdens_fitdata.csv'

fit_df = pd.read_csv(lowdens_dir / fit_data_filename, index_col=0)
fit_df

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,0.034400,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,0.013990,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,0.004756,0.003758,-3.484159,0.003447
3,angled,4.0,1.023927,0.018306,14.362077,0.343829,0.012245,0.007634,-3.423099,0.007020
4,angled,5.0,1.013085,0.010332,18.032248,0.256156,0.010595,0.004569,-3.457255,0.004239
...,...,...,...,...,...,...,...,...,...,...
83,sprobe,30.0,1.033611,0.010399,62.740506,0.552424,0.012586,0.002549,-2.732093,0.002039
84,sprobe,45.0,1.047113,0.014941,83.666200,1.018863,0.007753,0.003510,-2.690400,0.002807
85,sprobe,60.0,1.006582,0.010704,86.495626,0.719475,0.016005,0.002336,-2.594291,0.002069
86,sprobe,75.0,0.996101,0.012401,87.772275,0.795372,0.017997,0.002498,-2.483807,0.002520


In [69]:
flush_df = fit_df.loc[fit_df['probe'] == 'flush']
flush_df = flush_df.loc[flush_df['angle'] <= 30].drop(20).rename(columns={'angle':'theta'})

flush_df['theta_rads'] = np.radians(flush_df['theta'])
flush_df['theta_p'] = 0.0
flush_df['recession'] = 0.0
flush_df['theta_p_rads'] = np.radians(flush_df['theta_p'])

flush_ds = flush_df.to_xarray().swap_dims({'index':'theta'})

## Let's do some plotting!

In [70]:
dummy_theta = np.linspace(0.00001, 45.0, 5000)
flush_calced_a = lpu.calc_sheath_expansion_param(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    c_1=0.5,
    c_2=0.6,
)


In [71]:
# cb_palette = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', '#EE6677', '#AA3377', '#BBBBBB']
sqplt_probe_ind = {
    'angled_flush': [1,0],
    'angled_recessed': [1,1],
    'flat_flush': [0,0],
    'flat_recessed': [0,1],
    'flat_flush_gapless': [0,0]
}

In [72]:
## Make floating potential, corrected voltage and child-Langmuir voltage part of the piecewise ds

floating_pots = []
v_corrs = []
v_cls = []
# piecewise_ds.sel(probe='flat_flush', voltage=slice(-30,-4))['current_i'].plot.line(x='voltage', hue='theta', ax=ax)

for theta in piecewise_ds.theta.values:
    pw_plot_ds = piecewise_ds.sel(probe='flat_flush', theta=theta).mean(['y', 'z'])
    
    V_f = pw_plot_ds.swap_dims({'voltage':'current'}).interp(current=0)['voltage']
    floating_pots.append(V_f)    
    
    voltage_corr = pw_plot_ds['voltage'].values - V_f.values
    voltage_cl = np.float_power(np.abs(voltage_corr), 0.75)
    
    v_corrs.append(voltage_corr)
    v_cls.append(voltage_cl)
    
piecewise_ds = piecewise_ds.assign({
    'voltage_corr': (['theta', 'voltage'], v_corrs), 
    'voltage_cl': (['theta', 'voltage'], v_cls), 
    'V_f': (['theta'], floating_pots)
})

In [73]:
fig, ax = plt.subplots()

fitter = fts.IonCurrentSEFitter()
sheath_exps = []
fit_datas = []
for theta in piecewise_ds.theta.values:
    pw_plot_ds = piecewise_ds.sel(probe='flat_flush', voltage=slice(-30,-7), theta=theta)
    
#     ax.plot(pw_plot_ds.voltage_cl, pw_plot_ds.current_i, label=theta)
    fit_data = fitter.fit(pw_plot_ds.voltage_cl, pw_plot_ds.current_i, sigma=pw_plot_ds.d_current_i)
    fit_data.plot(ax=ax)
    
    sheath_exps.append(fit_data.get_sheath_exp())
    fit_datas.append(fit_data)
    
ax.set_xlabel(r'$|V|^{3/4}$')
ax.set_ylabel(r'$I_i$')
ax.legend(title=r'$\theta$')

In [26]:
probe='flat_flush'
old_plot_ds = old_combined_ds.sel(probe=probe)
plot_ds = combined_ds.sel(probe=probe)
fflwp_ds = floating_ds.sel(probe=probe)

fig, ax = plt.subplots()

# plot_ds['str_iv_a'].plot.line(x='theta', ax=ax, marker='x', mfc='none', linestyle='none', label='Straight')
# plot_ds['norm_iv_a'].plot.line(x='theta', ax=ax, marker='s', mfc='none', linestyle='none', label='Norm IV')

# plot_ds['ion_a'].plot.line(x='theta', ax=ax, marker='o', mfc='none', linestyle='none', label='Halfgrid')
# old_plot_ds['ion_a'].plot.line(x='theta', ax=ax, marker='o', mfc='none', linestyle='none', label='Wholegrid')

ax.errorbar(plot_ds.theta, plot_ds.ion_a, yerr=plot_ds.ion_d_a, marker='o', mfc='none', linestyle='none', label=r'Halfgrid - $V_w = 0$')
ax.errorbar(old_plot_ds.theta, old_plot_ds.ion_a, yerr=old_plot_ds.ion_d_a*4, marker='o', mfc='none', linestyle='none', label='Wholegrid')
# ax.errorbar(plot_ds.theta, plot_ds.norm_iv_a, yerr=plot_ds.norm_iv_d_a, marker='o', mfc='none', linestyle='none', label='HG Norm IV')

ax.errorbar(fflwp_ds.theta, fflwp_ds.ion_a, yerr=fflwp_ds.ion_d_a, marker='s', mfc='none', linestyle='none', label='Wholegrid - fflwp')


calced_a = lpu.calc_new_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    plot_ds.recession.values, plot_ds.theta_p_rads.values, 
#     c_1=0.5, c_2=0.6,
#     c_1=1.4, c_2=0.39, # from hg-theta=15-30
    c_1=2.0, c_2=0.14, # from hg-theta=11-30
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - $\theta_{large}$', fmt='-', 
            color='r', linewidth=0.8, alpha=0.6)

calced_a = lpu.calc_new_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    plot_ds.recession.values, plot_ds.theta_p_rads.values, 
    c_1=0.47, c_2=-0.0039, # from hg-theta=2-5
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted - $\theta_{small}$', fmt='-', 
            color='purple', linewidth=0.8, alpha=0.6)

# pw_thetas = [8.0, 10.0, 12.0]
# pw_as = [0.0132, 0.0183, 0.0212]
# ax.plot(pw_thetas, pw_as, 'x', label='Piecewise')
# ax.plot(piecewise_ds.theta.values, sheath_exps, 'x', label='Piecewise')


ax.set_ylim(1e-3, 5e-2)
ax.set_xlim(-0.5, 30.5)
ax.set_ylabel(r'$a$ [unitless]')
ax.set_xlabel(r'$\theta$ [$^{\circ}$]')

ax.legend()
fig.tight_layout()

In [28]:
probe='flat_flush'
fflwp_ds = floating_ds.sel(probe=probe)
fixed2_ds = fwp2_ds.sel(probe=probe)

fig, ax = plt.subplots()

ax.errorbar(fflwp_ds.theta, fflwp_ds.ion_a, yerr=fflwp_ds.ion_d_a, marker='x', mfc='none', linestyle='none', label=r'$V_w = V_{fl} \sim 4$')
# ax.errorbar(old_plot_ds.theta, old_plot_ds.ion_a, yerr=old_plot_ds.ion_d_a*4, marker='+', mfc='none', linestyle='none', label=r'$V_w=0$')
ax.errorbar(fixed2_ds.theta, fixed2_ds.ion_a, yerr=fixed2_ds.ion_d_a, marker='x', mfc='none', linestyle='none', label=r'$V_w=2$')

ax.errorbar(piecewise_ds.theta.values, sheath_exps, fmt='s', mfc='none', label=r'$V_w=0 (Piecewise)$')


calced_a = lpu.calc_new_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    plot_ds.recession.values, plot_ds.theta_p_rads.values, 
    c_1=1.5, c_2=0.63,
#     c_1=1.4, c_2=0.39, # from hg-theta=15-30
#     c_1=2.0, c_2=0.14, # from hg-theta=11-30
)
ax.errorbar(dummy_theta, calced_a, label=r'Predicted', fmt='-', 
            color='r', linewidth=0.8, alpha=0.6)

ax.set_ylim(1e-3, 5e-2)
ax.set_xlim(-0.5, 30.5)
ax.set_ylabel(r'$a \;\; [unitless]$')
ax.set_xlabel(r'$\theta \;\; [^\circ]$')

ax.legend()
fig.tight_layout()

In [29]:


ff_hg_ds = combined_ds.sel(probe='flat_flush')
ff_wg_ds = old_combined_ds.sel(probe='flat_flush')
ar_hg_ds = halfgrid_ar_ds.sel(probe='angled_recessed')
ar_wg_ds = old_combined_ds.sel(probe='angled_recessed')

fig, ax = plt.subplots(2, sharex=True, sharey=True, figsize=[6,10])

ax[0].errorbar(ff_wg_ds.theta, ff_wg_ds.ion_a, yerr=ff_wg_ds.ion_d_a, marker='o', mfc='none', 
               linestyle='none', label=r'Wholegrid - $V_w = 0$')
ax[0].errorbar(ff_hg_ds.theta, ff_hg_ds.ion_a, yerr=ff_hg_ds.ion_d_a, marker='o', mfc='none', 
               linestyle='none', label=r'Halfgrid - $V_w = 0$')

calced_a = lpu.calc_new_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    ff_hg_ds.recession.values, ff_hg_ds.theta_p_rads.values, 
#     c_1=0.5, c_2=0.6,
#     c_1=1.4, c_2=0.39, # from hg-theta=15-30
    c_1=2.0, c_2=0.14, # from hg-theta=11-30
)
ax[0].errorbar(dummy_theta, calced_a, label=r'Predicted - $\theta_{large}$', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)

calced_a = lpu.calc_new_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    ff_hg_ds.recession.values, ff_hg_ds.theta_p_rads.values, 
    c_1=0.47, c_2=-0.0039, # from hg-theta=2-5
)
ax[0].errorbar(dummy_theta, calced_a, label=r'Predicted - $\theta_{small}$', fmt='-', 
               color='purple', linewidth=0.8, alpha=0.6)


ax[1].errorbar(ar_wg_ds.theta, ar_wg_ds.ion_a, yerr=ar_wg_ds.ion_d_a, marker='o', 
               mfc='none', linestyle='none', label=r'Wholegrid - $V_w = 0$')
ax[1].errorbar(ar_hg_ds.theta, ar_hg_ds.ion_a, yerr=ar_hg_ds.ion_d_a, marker='o', 
               mfc='none', linestyle='none', label=r'Halfgrid - $V_w = 0$')

calced_a_ar = lpu.calc_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
#     ar_hg_ds.recession.values, ar_hg_ds.theta_p_rads.values, 
    c_1=2.2, c_2=-0.058, # from hg-theta=11-30
)
ax[1].errorbar(dummy_theta, calced_a_ar, label=r'Predicted - $\theta_{large}$', fmt='-', 
               color='red', linewidth=0.8, alpha=0.6)

calced_a_ar = lpu.calc_new_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    ar_hg_ds.recession.values, ar_hg_ds.theta_p_rads.values, 
    c_1=1.5, c_2=-0.52, # from hg-theta=2-5
)
ax[1].errorbar(dummy_theta, calced_a_ar, label=r'Predicted - $\theta_{small}$', fmt='-', 
               color='purple', linewidth=0.8, alpha=0.6)


ax[0].set_ylim(1e-3, 4e-2)
ax[0].set_xlim(-0.5, 30.5)
ax[0].set_ylabel(r'$a$ [unitless]')
ax[1].set_ylabel(r'$a$ [unitless]')
ax[1].set_xlabel(r'$\theta$ [$^{\circ}$]')

ax[0].legend()
ax[1].legend()
fig.tight_layout()

AttributeError: 'Dataset' object has no attribute 'theta_p_rads'

## $I_{sat}$ Plots

In [30]:
# piecewise_ds['V_f'].plot.line(x='theta', marker='x', ls='none')
# print(plot_ds)
fig, ax = plt.subplots(5, sharex=True, figsize=[6,10])
labels = [r'$V_w = 0$ (halfgrid)', r'$V_w = 0$', r'$V_w = 2$', r'$V_w = V_{fl} \sim 4$']

for i, ds in enumerate([plot_ds, old_plot_ds, fwp2_ds, fflwp_ds]):
    ds.str_iv_V_f.plot.line(x='theta', ax=ax[0], label=labels[i])
    ds.str_iv_a.plot.line(x='theta', ax=ax[1], label=labels[i])
    ds.str_iv_T_e.plot.line(x='theta', ax=ax[2], label=labels[i])
    ds.str_iv_I_sat.plot.line(x='theta', ax=ax[3], label=labels[i])
    ds.str_iv_reduced_chi2.plot.line(x='theta', ax=ax[4], label=labels[i])

for axis in ax:
    axis.legend()


In [31]:
fig, ax = plt.subplots()
plot_ds.str_iv_V_f.plot.line(x='theta')
plot_ds.str_iv_V_f.mean()

<xarray.DataArray 'str_iv_V_f' ()>
array(-2.45630705)
Coordinates:
    theta_p       float64 0.0
    theta_p_rads  float64 0.0
    probe         <U10 'flat_flush'

In [32]:
alpha = np.radians(10.0)
flush_square = lpu.AngledTipProbe(a=5e-3, b=5e-3, L=5e-3, g=1e-3, d_perp=0e-4, 
                                  theta_f=0.0, theta_p=0.0)

probe_I_2d = lpu.sound_speed(T_e) * n_0 * c.ELEM_CHARGE * ((L + g) * lambda_D) * np.sin(alpha) * flush_square.a
alt_probe_I_2d = flush_square.get_isat(T_e, n_0, alpha) #/ flush_square.a
print(probe_I_2d, alt_probe_I_2d)

0.025764018536306915 [0.02583201]


In [33]:
fflwp_ds['denorm_current'] = denormaliser(fflwp_ds.current, 'current')
fflwp_ds['denorm_d_current'] = denormaliser(fflwp_ds.d_current, 'current')


In [34]:
def denorm(arr):
    return denormaliser(arr, 'current')

In [35]:
fig, ax = plt.subplots(2, figsize=[10,10], sharex=True)
labels = [r'$V_w = 0$ (halfgrid)', r'$V_w = 0$', r'$V_w = 2$', r'$V_w = V_{fl} \sim 4$']
denorms = [denormaliser, denormaliser_coarse, denormaliser_coarse, denormaliser_coarse]

for i, ds in enumerate([plot_ds, old_plot_ds, fwp2_ds, fflwp_ds]):
    if 'probe' in ds.dims:
        ds = ds.sel(probe='flat_flush')
    isat = denorms[i](-ds.str_iv_I_sat, 'current').values * flush_square.a
    d_isat = denorms[i](ds.str_iv_d_I_sat, 'current').values * flush_square.a
    ax[0].errorbar(ds.theta.values, isat, yerr=d_isat, 
                   label=labels[i])
    
    ds.str_iv_a.plot.line(x='theta', marker='x', ls='none', ax=ax[1], label=labels[i])


calced_I_sat = flush_square.get_isat(T_e, n_0, np.radians(dummy_theta))
calced_I_sat_alt = lpu.sound_speed(T_e) * n_0 * c.ELEM_CHARGE * ((L + g) * lambda_D) * np.sin(np.radians(dummy_theta)) * flush_square.a
ax[0].errorbar(dummy_theta, calced_I_sat, label=r'probe_def', fmt='-', color='b', linewidth=0.8, alpha=0.6)
ax[0].errorbar(dummy_theta, calced_I_sat_alt, label=r'$I_{sat}$ def', fmt='-', color='r', linewidth=0.8, alpha=0.6)

ax[0].set_xlim(0, 30.5)
ax[0].set_ylim(0)#, 15.0773)
ax[0].legend()


In [36]:
labels = [r'$V_w = 0$ (halfgrid)', r'$V_w = 0$', r'$V_w = 2$', r'$V_w = V_{fl} \sim 4$']
denorms = [denormaliser, denormaliser_coarse, denormaliser_coarse, denormaliser_coarse]

fig, ax = plt.subplots(4, len(denorms), figsize=[15,15], sharex=True)


for i, ds in enumerate([plot_ds, old_plot_ds, fwp2_ds, fflwp_ds]):
    if 'probe' in ds.dims:
        ds = ds.sel(probe='flat_flush')
    isat = denorms[i](-ds.str_iv_I_sat, 'current').values * flush_square.a
    d_isat = denorms[i](ds.str_iv_d_I_sat, 'current').values * flush_square.a
    
    ideal_I_sat = flush_square.get_isat(T_e, n_0, ds.theta_rads.values)
    
    ax[0,i].errorbar(ds.theta.values, isat, yerr=d_isat, label=labels[i])
    ax[1,i].errorbar(ds.theta.values, isat / ideal_I_sat, yerr=d_isat / ideal_I_sat, label=labels[i])
    
    ax[2,i].plot(ds.theta.values, (isat / ideal_I_sat) * ds.ion_a.values, label=labels[i])
    
    ds.str_iv_a.plot.line(x='theta', marker='x', ls='none', ax=ax[3,i], label=labels[i])

    calced_I_sat = flush_square.get_isat(T_e, n_0, np.radians(dummy_theta))
    ax[0,i].errorbar(dummy_theta, calced_I_sat, label=r'probe_def', fmt='-', color='b', linewidth=0.8, 
                     alpha=0.6)


    ax[0,i].set_xlim(0, 30.5)
    ax[0,i].set_ylim(0)#, 15.0773)
    ax[0,i].legend()
# fig.tight_layout()

# Comparing the I_sat overestimate to the fitted a value doesn't seem to show anything

In [37]:
probe='flat_flush'
fflwp_ds = floating_ds.sel(probe=probe)

fig, ax = plt.subplots(2, sharex=True)
ax[0].errorbar(fflwp_ds.theta, denorm(fflwp_ds.ion_I_sat * flush_square.a), 
               yerr=denorm(fflwp_ds.ion_d_I_sat * flush_square.a), 
               marker='s', mfc='none', linestyle='none', label='Wholegrid - fflwp')

# ax[0].errorbar(old_plot_ds.theta, denormaliser(old_plot_ds.ion_I_sat, 'current'), 
#                yerr=denormaliser(old_plot_ds.ion_d_I_sat, 'current'),  
#                marker='o', mfc='none', linestyle='none', label='Wholegrid')

calced_I_sat = flush_square.get_isat(T_e, n_0, np.radians(dummy_theta))
ax[0].errorbar(dummy_theta, calced_I_sat, label=r'calc', fmt='-', 
               color='r', linewidth=0.8, alpha=0.6)

calced_comp_I_sat = flush_square.get_isat(T_e, n_0, np.radians(fflwp_ds.theta.values))
ax[1].plot(fflwp_ds.theta.values, calced_comp_I_sat / denorm(fflwp_ds.ion_I_sat * flush_square.a))

ax[0].set_ylabel(r'$I_{sat}$ [A/m]')
ax[1].set_xlabel(r'$\theta$')

ax[0].set_xlim(-0.5, 30.5)
ax[0].set_ylim(0)

ax[0].legend()
fig.tight_layout()

In [38]:
fig, axes = plt.subplots(2, 2, figsize=[10,6])

for i, probe in enumerate(old_combined_ds.probe.values):
    plot_ds = old_combined_ds.sel(probe=probe)

    ii, jj = sqplt_probe_ind[probe]
    
    ax = axes[ii, jj]

    # plot_ds['str_iv_a'].plot.line(x='theta', ax=ax, marker='x', mfc='none', linestyle='none', label='Straight')
    # plot_ds['norm_iv_a'].plot.line(x='theta', ax=ax, marker='s', mfc='none', linestyle='none', label='Norm IV')
#     plot_ds['ion_a'].plot.line(x='theta', ax=ax, marker='o', mfc='none', linestyle='none', label='Ion')
    ax.errorbar(plot_ds['theta'].values, plot_ds['ion_a'].values, yerr=plot_ds['ion_d_a'].values, marker='o', mfc='none', linestyle='none', 
                label=f'{probe}', color=cb_palette[2*i], ecolor=cb_palette[2*i])

    calced_a = lpu.calc_new_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values, c_1=0.5, c_2=0.6,
    )

#     ax.errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', color='r', linewidth=0.8, alpha=0.6)
#     # ax.errorbar(dummy_theta, flush_calced_a, label=r'calc', fmt='-', color='b', linewidth=0.8, alpha=0.6)
#     ax.set_ylim(1e-3, 5e-2)
#     ax.set_xlim(-0.5, 30.5)

    ax.legend()
fig.tight_layout()

In [39]:
fig, ax = plt.subplots()

# probes_oi = [p for p in probe_desig.keys() if 'recessed' not in p]
# probes_oi = [p for p in probe_desig.keys() if 'flat' in p]
# probes_oi = [p for p in probe_desig.keys()]
probes_oi = ['flat_flush']


for i, probe in enumerate(probes_oi):
    plot_ds = combined_ds.sel(probe=probe)

#     plot_ds['str_iv_a'].plot.line(x='theta', ax=ax, marker='x', mfc='none', linestyle='none', label='Straight')
#     plot_ds['norm_iv_a'].plot.line(x='theta', ax=ax, marker='s', mfc='none', linestyle='none', label=f'{probe} - norm',
#                                    color=cb_palette[2*i])
#     plot_ds['ion_a'].plot.line(x='theta', ax=ax, marker='o', mfc='none', linestyle='none', label=f'{probe} - ion', 
#                                color=cb_palette[2*i])
    ax.errorbar(plot_ds['theta'].values, plot_ds['ion_a'].values, yerr=plot_ds['ion_d_a'].values, marker='o', mfc='none', linestyle='none', 
                label=f'{probe}', color=cb_palette[2*i], ecolor=cb_palette[2*i])

    calced_a = lpu.calc_new_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values, c_1=0.9, c_2=0.9,
    )

    ax.errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', color=cb_palette[2*i], linewidth=0.8, alpha=0.6)
    # ax.errorbar(dummy_theta, flush_calced_a, label=r'calc', fmt='-', color='b', linewidth=0.8, alpha=0.6)
    ax.set_ylim(1e-3, 2.5e-2)
    ax.set_xlim(-0.5, 30.5)

    ax.legend()

In [40]:
fig, axes = plt.subplots(2, 2, figsize=[8,8], sharex=True)

# probes_oi = [p for p in probe_desig.keys()]
probes_oi = ['flat_flush']


probe_old_cs = [
    (0.09, 5.6),
    (-0.031, 5.2),
    (0.44, 6.3),
    (3.8, -4.0)
]
probe_new_cs = [
    (-0.24, 3.5),
    (0.76, 2.3),
    (0.8, 3.9),
    (3, 0.5)
]
probe_2db_cs = [
    (1, 3.9),
    (2.4, 1.9),
    (0.44, 6.3),
    (3.8, -4.0)
]


for i, probe in enumerate(probes_oi[:-1]):
    ax = axes[i%2][i//2]
    plot_ds = combined_ds.sel(probe=probe)

    ax.errorbar(plot_ds['theta'].values, plot_ds['ion_a'].values, yerr=plot_ds['ion_d_a'].values, marker='o', mfc='none', linestyle='none', 
                label=f'{probe}', color=cb_palette[i], ecolor=cb_palette[i])
#     c_1 = 0.9
#     c_2 = 0.6

#     c_1 = 6
#     c_2 = 0.6

#     c_1 = 2.4
#     c_2 = 1.9

#     c_1 = 0.76
#     c_2 = 2.3

    c_2_o, c_1_o = probe_old_cs[i]
    c_2_n, c_1_n = probe_new_cs[i]
    c_2_b, c_1_b = probe_2db_cs[i]
    calced_a = lpu.calc_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        c_1=c_1_o, c_2=c_2_o
    )
    
    calced_a_new = lpu.calc_new_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values,
        c_1=c_1_n, c_2=c_2_n
    )
    
    calced_a_2db = lpu.calc_2d_box_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values,
        c_1=c_1_b, c_2=c_2_b
    )

    ax.errorbar(dummy_theta, calced_a, label=r'old', fmt='-', color=cb_palette[5], linewidth=0.8, alpha=0.6)
    ax.errorbar(dummy_theta, calced_a_new, label=r'new', fmt='-', color=cb_palette[6], linewidth=0.8, alpha=0.6)
    ax.errorbar(dummy_theta, calced_a_2db, label=r"pete's", fmt='-', color=cb_palette[7], linewidth=0.8, alpha=0.6)
    ax.set_ylim(1e-3, 3.5e-1)
    ax.set_xlim(-0.5, 30.5)
#     ax.set_title(r'$c_1$ = {}, $c_2$ = {}'.format(c_1, c_2))

    ax.legend()

axes[1,1].set_ylim(1e-3, 5e-1)
# fig.suptitle(r'$c_1$ = {}, $c_2$ = {}'.format(c_1, c_2))
    
fig.tight_layout(rect=[0, 0, 1, 0.97])

In [41]:
fig, axes = plt.subplots(3, figsize=[6,8])

sheath_exp_funcs = [
    lpu.calc_sheath_expansion_param,
    lpu.calc_new_sheath_expansion_param,
    lpu.calc_2d_box_sheath_expansion_param,
]

# probes_oi = [p for p in probe_desig.keys()]
probes_oi = ['flat_flush']

for j, ax in enumerate(axes):
    for i, probe in enumerate(probes_oi):
        plot_ds = combined_ds.sel(probe=probe)

        ax.errorbar(plot_ds['theta'].values, plot_ds['ion_a'].values, yerr=plot_ds['ion_d_a'].values, marker='o', mfc='none', linestyle='none', 
                    label=f'{probe}', color=cb_palette[2*i], ecolor=cb_palette[2*i])
        
        if j == 0:
            calced_a = sheath_exp_funcs[j](
                T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
                c_1=0.9, c_2=0.9
            )
        else:
            calced_a = sheath_exp_funcs[j](
                T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
                plot_ds.recession.values, plot_ds.theta_p_rads.values, c_1=0.9, c_2=0.9,
            )

        ax.errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', color=cb_palette[2*i], linewidth=0.8, alpha=0.6)
        # ax.errorbar(dummy_theta, flush_calced_a, label=r'calc', fmt='-', color='b', linewidth=0.8, alpha=0.6)
        ax.set_ylim(1e-3, 2.5e0)
        ax.set_xlim(-0.5, 30.5)

        ax.legend()
        
fig.tight_layout()

In [74]:
probe_inputs = []
for i in range(5):
    spl_path = spice_dir / all_run_dirs[scans[i - 2]][0]
    probe_inputs.append(next(next(spl_path.glob('backup*')).glob('input.inp')))
    
# probe_inputs

fig, axes = plt.subplots(2, 3, figsize=[8,4], sharex=True, sharey=True)

for i, probe_inp in enumerate(probe_inputs):
    ax = axes[i%2][i//2]
    spu.plot_2d_sim_window(probe_inp, ax=ax, colour=cb_palette[i])
    ax.set_ylim(0,222)
    ax.set_xlim(0,555) #759)
    
fig.tight_layout()

{'circle': [], 'triangle': [], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
{'circle': [], 'triangle': [], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
{'circle': [], 'triangle': [<Section: triangle0>], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}
{'circle': [], 'triangle': [], 'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>]}


---
## Normalised IV plots

In [43]:
def a_0(theta):
    return lpu.calc_sheath_expansion_param(
        T_e, 
        n_0, 
        L * lambda_D, 
        g * lambda_D, 
        np.radians(theta),
        c_1=0.5, 
        c_2=0.6
    )

In [44]:
fig, ax = plt.subplots(2, sharex=True, figsize=[6,10])

# probes_oi = [p for p in probe_desig.keys() if 'recessed' not in p]
probes_oi = ['flat_flush']#, 'angled_flush']

for i, probe in enumerate(probes_oi):
    dummy_voltage = np.linspace(-30, 0, 9950)
    piv_fitter = fts.PartialIVFitter()
    ax[i].plot(dummy_voltage, -piv_fitter.fit_function(dummy_voltage, *[1.0, a_0(90.0), 1.0]), 
               color='k', linestyle='--', label=f'Ideal')
    ax[i].plot(dummy_voltage, -piv_fitter.fit_function(dummy_voltage, *[1.0, a_0(10.0), 1.0]), 
               color='silver', linestyle='--', alpha=0.7, label=f'Ideal 10')
    
    for theta in combined_ds.theta.values[1::2]:
        plot_ds = combined_ds.sel(probe=probe, theta=theta)

        V_f = fts.IVFitter.find_floating_pot(plot_ds['voltage'][:-20], plot_ds['current'][:-20])
        print(f'{theta}:{V_f}')
        plot_ds = plot_ds.sel(voltage=slice(-30.0, V_f))

        voltage = (plot_ds['voltage'].values - V_f)[::2] * T_e
        current = denormaliser(plot_ds['current'].values[::2], 'current')
        d_current = denormaliser(plot_ds['d_current'].values[::2], 'current')
        
#         I_0 = plot_ds.norm_iv_I_sat.values
        I_0 = lpu.sound_speed(T_e) * n_0 * c.ELEM_CHARGE * ((L + g) * lambda_D) * np.sin(np.radians(theta))
        theta_label = plot_ds.theta_p.values + theta
        
        print(voltage.size, current.size, d_current.size, I_0.size)
        ax[i].errorbar(voltage, current/I_0, yerr=d_current/I_0, fmt='.', mfc='none', markersize=5,) 
#                        label=f'{theta_label} - a={plot_ds.ion_a.values:.2g}')
        ax[i].legend()
        ax[i].set_title(probe) 
        
        ax[i].set_ylim(-3.0,0.0)
        ax[i].set_xlim(-30.0,0.0)

fig.tight_layout()

3.0:-2.3333989113934814
553 553 553 1
5.0:-2.33974386725363
553 553 553 1


ValueError: attempt to get argmin of an empty sequence

In [21]:
plot_ds

<xarray.Dataset>
Dimensions:               (theta: 2, voltage: 1159)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 ... -1.05 -1.025
  * theta                 (theta) float64 6.0 6.0
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.1047 0.1047
    probe                 <U10 'flat_flush'
Data variables:
    str_iv_chi2           (theta) float64 846.9 736.8
    str_iv_reduced_chi2   (theta) float64 0.7595 0.7181
    str_iv_I_sat          (theta) float64 -44.36 -44.27
    str_iv_d_I_sat        (theta) float64 0.05433 0.06072
    str_iv_a              (theta) float64 0.00423 0.004502
    str_iv_d_a            (theta) float64 0.0001579 0.0001596
    str_iv_T_e            (theta) float64 1.11 1.095
    str_iv_d_T_e          (theta) float64 0.005812 0.03708
    str_iv_V_f            (theta) float64 -2.347 -2.375
    str_iv_d_V_f          (theta) float64 0.003016 0.07659
    str_iv_run_dir        (theta) object 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-6.0' 'marconi/spice2/sheath_exp_hg/flat_flush_gapfill/alpha_yz_-6.0'
    str_iv_voltage_min    (theta) float64 -29.98 -29.98
    str_iv_voltage_max    (theta) float64 -2.025 -4.25
    norm_iv_chi2          (theta) float64 1.192e+03 2.381e+03
    norm_iv_reduced_chi2  (theta) float64 1.08 2.316
    norm_iv_I_sat         (theta) float64 -43.68 -42.63
    norm_iv_d_I_sat       (theta) float64 0.05126 0.08395
    norm_iv_a             (theta) float64 0.005693 0.00818
    norm_iv_d_a           (theta) float64 0.0001558 0.0002505
    norm_iv_voltage_min   (theta) float64 -27.63 -28.98
    norm_iv_voltage_max   (theta) float64 -0.005394 -3.25
    ion_chi2              (theta) float64 618.7 652.3
    ion_reduced_chi2      (theta) float64 0.7161 0.7113
    ion_I_sat             (theta) float64 44.4 44.09
    ion_d_I_sat           (theta) float64 0.06237 0.05883
    ion_a                 (theta) float64 0.003802 0.004418
    ion_d_a               (theta) float64 0.0001704 0.0001579
    ion_voltage_min       (theta) float64 3.836 3.846
    ion_voltage_max       (theta) float64 12.05 12.49
    current               (theta, voltage) float64 -46.87 -47.77 ... nan nan
    current_e             (theta, voltage) float64 0.0 0.0 0.0 ... nan nan nan
    current_i             (theta, voltage) float64 46.87 47.77 48.04 ... nan nan
    d_current             (theta, voltage) float64 0.6242 0.5604 ... nan nan
    d_current_e           (theta, voltage) float64 0.0 0.0 0.0 ... nan nan nan
    d_current_i           (theta, voltage) float64 0.6242 0.5604 ... nan nan
    time                  (theta, voltage) float64 0.0 0.0214 ... nan nan
    recession             float64 0.0

---
### Decomposition of sheath expansion param plots
These can be categorised into 3 modes:
1. The original formula from Sam/Bergmann (no label)
2. My rederivation (new)
3. Pete's 2d box derivation (2d box)


In [45]:
decomp_modes = [
    None, 
    lpu.decompose_sheath_exp_param, 
    lpu.decompose_new_sheath_exp_param, 
    lpu.decompose_2d_box_sheath_exp_param,
    lpu.decompose_alt_new_sheath_exp_param
]
mode_labels = [
    None, 
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}\cdot[L + g]$'),
    (r'$\tan{\theta} + 2\tan{\theta_p}$', r'$a\sin^{1/2}{\theta}[(L+g)\tan{\theta} + L\tan{\theta_p} - d_{\perp}]$'),
    (r'$\cot{\theta_{tot}}$', r'$a\sin^{1/2}{\theta_{tot}}[L_{eff} + \Delta_0 \cot{\theta_{tot}}]$'),
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}[L + g + (L\tan{\theta_p} - d_{\perp}) * \cot{\theta}]$'),
]
mode_constants = [
    None,
    ('y_0', 'm'),
    ('m', 'y_0'),
    ('y_0', 'm'),
    ('y_0', 'm'),
]

print(f'L = {L} \n g = {g}')

def plot_decomp(ds, sheath_label='ion_a', mode=1, fit_fl=True, ax=None, kwargs_for_plot={}, 
                kwargs_for_fitplot={}, colour='r', plot_label=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    
    if mode in [1, 2, 3, 4]:
        x, y = decomp_modes[mode](ds[sheath_label], ds['theta_rads'], L, g, 
                                  ds['recession'], ds['theta_p_rads'])
    else:
        raise ValueError('Mode must be 1, 2 or 3')
    
    x = x[~np.isnan(y)]
    y = y[~np.isnan(y)]
    
    if plot_label is None:
        plot_label = sheath_label
    
    ax.errorbar(x, y, yerr=ds[sheath_label]*y, color=colour, label=plot_label, **kwargs_for_plot)
    
    sl_fitter = fts.StraightLineFitter()
    fit_data = sl_fitter.fit(x, y)
    if fit_fl:    
        c1, c2 = mode_constants[mode]
        fit_label = r'$c_1$ = {:.2g}, $c_2$ = {:.2g}'.format(fit_data.get_param(c1), fit_data.get_param(c2))
        ax.plot(*fit_data.get_fit_plottables(), color=colour, label=fit_label, 
                **kwargs_for_fitplot)
        
    ax.set_xlabel(mode_labels[mode][0])
    ax.set_ylabel(mode_labels[mode][1])
    ax.legend()
    
    return ax, fit_data
        

L = 300.0 
 g = 60.0


In [46]:
combined_ds

<xarray.Dataset>
Dimensions:               (probe: 1, theta: 15, voltage: 1201)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 ... 1.665e-15 0.025
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.3491 0.5236
  * probe                 (probe) <U10 'flat_flush'
Data variables:
    str_iv_chi2           (probe, theta) float64 723.8 802.2 ... 1.067e+03
    str_iv_reduced_chi2   (probe, theta) float64 0.6514 0.7195 ... 0.9598 0.9566
    str_iv_I_sat          (probe, theta) float64 -14.44 -21.82 ... -108.4 -150.9
    str_iv_d_I_sat        (probe, theta) float64 0.01673 0.02617 ... 0.2316
    str_iv_a              (probe, theta) float64 0.005932 0.005801 ... 0.008279
    str_iv_d_a            (probe, theta) float64 0.0001485 ... 0.0001997
    str_iv_T_e            (probe, theta) float64 1.113 1.099 ... 1.082 1.076
    str_iv_d_T_e          (probe, theta) float64 0.005743 0.005632 ... 0.00541
    str_iv_V_f            (probe, theta) float64 -2.368 -2.338 ... -2.666 -2.736
    str_iv_d_V_f          (probe, theta) float64 0.003084 0.002892 ... 0.002625
    str_iv_run_dir        (probe, theta) object 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (probe, theta) float64 -29.98 -29.98 ... -29.98 -29.98
    str_iv_voltage_max    (probe, theta) float64 -2.125 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (probe, theta) float64 1.106e+03 ... 1.176e+03
    norm_iv_reduced_chi2  (probe, theta) float64 1.003 0.9827 ... 1.051 1.081
    norm_iv_I_sat         (probe, theta) float64 -14.2 -21.51 ... -107.3 -149.3
    norm_iv_d_I_sat       (probe, theta) float64 0.01626 0.02425 ... 0.2018
    norm_iv_a             (probe, theta) float64 0.007396 0.007055 ... 0.009172
    norm_iv_d_a           (probe, theta) float64 0.0001542 ... 0.0001874
    norm_iv_voltage_min   (probe, theta) float64 -27.61 -27.64 ... -27.29 -27.25
    norm_iv_voltage_max   (probe, theta) float64 -0.01163 -0.0166 ... -0.0208
    ion_chi2              (probe, theta) float64 528.0 593.0 ... 820.2 776.8
    ion_reduced_chi2      (probe, theta) float64 0.6118 0.6864 ... 0.9649 0.916
    ion_I_sat             (probe, theta) float64 14.46 21.91 ... 108.5 150.8
    ion_d_I_sat           (probe, theta) float64 0.01913 0.03042 ... 0.2797
    ion_a                 (probe, theta) float64 0.005252 0.004938 ... 0.007918
    ion_d_a               (probe, theta) float64 0.0001623 ... 0.0002357
    ion_voltage_min       (probe, theta) float64 3.839 3.842 ... 3.843 3.844
    ion_voltage_max       (probe, theta) float64 12.05 12.06 ... 11.94 11.93
    current               (probe, theta, voltage) float64 -15.88 -15.98 ... nan
    current_e             (probe, theta, voltage) float64 0.0 0.0 ... 0.0 nan
    current_i             (probe, theta, voltage) float64 15.88 15.98 ... nan
    d_current             (probe, theta, voltage) float64 0.1944 0.2147 ... nan
    d_current_e           (probe, theta, voltage) float64 0.0 0.0 ... 0.0 nan
    d_current_i           (probe, theta, voltage) float64 0.1944 0.2147 ... nan
    time                  (probe, theta, voltage) float64 0.0 0.06436 ... nan
    recession             float64 0.0

In [47]:
probe = 'flat_flush'

plot_ds = combined_ds.sel(probe=probe, theta=slice(11, 30))
plot_whole_ds = combined_ds.sel(probe=probe)
plot_f_ds = flush_ds.sel(theta=slice(3,30))

kwargs_for_plot={
    'ls':'none', 
    'mfc':'none', 
    'marker': '.',
}
kwargs_for_fitplot={
    'ls':'none', 
    'mfc':'none', 
}

fig, ax = plt.subplots(3, figsize=[6,10])
_, fit_data_1i = plot_decomp(plot_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
_, fit_data_1a = plot_decomp(plot_f_ds, sheath_label='a', mode=1, ax=ax[0], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)

_, fit_data_2i = plot_decomp(plot_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
_, fit_data_2a = plot_decomp(plot_f_ds, sheath_label='a', mode=2, ax=ax[1], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)

_, fit_data_4i = plot_decomp(plot_ds, sheath_label='ion_a', mode=4, ax=ax[2], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
_, fit_data_4a = plot_decomp(plot_f_ds, sheath_label='a', mode=4, ax=ax[2], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=4, ax=ax[2], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)

fig.tight_layout()


In [48]:
probe = 'flat_flush'

plot_ds = floating_ds.sel(probe=probe, theta=slice(8, 30))
plot_whole_ds = floating_ds.sel(probe=probe)
plot_f_ds = flush_ds.sel(theta=slice(3,30))

kwargs_for_plot={
    'ls':'none', 
    'mfc':'none', 
    'marker': '.',
}
kwargs_for_fitplot={
    'ls':'none', 
    'mfc':'none', 
}

fig, ax = plt.subplots(2, figsize=[6,10])
plot_decomp(plot_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_f_ds, sheath_label='a', mode=1, ax=ax[0], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)

plot_decomp(plot_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_f_ds, sheath_label='a', mode=2, ax=ax[1], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)


fig.tight_layout()

In [49]:
probe = 'angled_recessed'

plot_ds = halfgrid_ar_ds.sel(probe=probe, theta=slice(5, 12))
plot_whole_ds = halfgrid_ar_ds.sel(probe=probe)
plot_wg_ds = old_combined_ds.sel(probe=probe, theta=slice(None, 12))

kwargs_for_plot={
    'ls':'none', 
    'mfc':'none', 
    'marker': '.',
}
kwargs_for_fitplot={
    'ls':'none', 
    'mfc':'none', 
}

fig, ax = plt.subplots(3, figsize=[6,15])
plot_decomp(plot_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[0], 
            kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)
plot_decomp(plot_wg_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[1], 
            kwargs_for_plot=kwargs_for_plot)

plot_decomp(plot_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[0], 
            kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)
plot_decomp(plot_wg_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[1], 
            kwargs_for_plot=kwargs_for_plot)

plot_decomp(plot_ds, sheath_label='ion_a', mode=3, ax=ax[2], colour=cb_palette[0], 
            kwargs_for_plot=kwargs_for_plot)
plot_decomp(plot_whole_ds, sheath_label='ion_a', mode=3, ax=ax[2], colour=cb_palette[2],
            kwargs_for_plot=kwargs_for_plot, fit_fl=False)
plot_decomp(plot_wg_ds, sheath_label='ion_a', mode=3, ax=ax[2], colour=cb_palette[1], 
            kwargs_for_plot=kwargs_for_plot)


fig.tight_layout()

KeyError: 'theta_rads'

In [56]:
print('Old Param (ion fit)')
print(f'c1 = {fit_data_1i.get_param("y_0"):.3g} ± {fit_data_1i.get_param_err("y_0"):.3g}')
print(f'c2 = {fit_data_1i.get_param("m"):.3g} ± {fit_data_1i.get_param_err("m"):.3g}')

print('\nNew Param (ion fit)')
print(f'c1 = {fit_data_2i.get_param("m"):.3g} ± {fit_data_2i.get_param_err("m"):.3g}')
print(f'c2 = {fit_data_2i.get_param("y_0"):.3g} ± {fit_data_2i.get_param_err("y_0"):.3g}')

print('\n\nOld Param (old norm iv fit)')
print(f'c1 = {fit_data_1a.get_param("y_0"):.3g} ± {fit_data_1a.get_param_err("y_0"):.3g}')
print(f'c2 = {fit_data_1a.get_param("m"):.3g} ± {fit_data_1a.get_param_err("m"):.3g}')

print('\nNew Param (old norm iv fit)')
print(f'c1 = {fit_data_2a.get_param("m"):.3g} ± {fit_data_2a.get_param_err("m"):.3g}')
print(f'c2 = {fit_data_2a.get_param("y_0"):.3g} ± {fit_data_2a.get_param_err("y_0"):.3g}')


Old Param (ion fit)
c1 = 3.49 ± 0.399
c2 = -0.145 ± 0.0397

New Param (ion fit)
c1 = 3.04 ± 0.296
c2 = -0.0671 ± 0.0756


Old Param (old norm iv fit)
c1 = 5.22 ± 0.515
c2 = 0.439 ± 0.0674

New Param (old norm iv fit)
c1 = 5.95 ± 0.419
c2 = 0.3 ± 0.118


## Looking at the temperature scan

In [38]:
def mod_pow(v, power=0.75, v_f=0):
    return np.float_power(np.abs(v - v_f), power)

In [64]:
tempscan_1_ds = xr.load_dataset('se_highV_tempscan_ivs.nc')
tempscan_2_ds = xr.load_dataset('se_lowV_tempscan_ivs.nc')
tempscan_1_ds

<xarray.Dataset>
Dimensions:               (temperature: 2, theta: 6, voltage: 1210)
Coordinates:
  * temperature           (temperature) float64 5.0 10.0
  * theta                 (theta) float64 2.0 3.0 5.0 8.0 10.0 30.0
  * voltage               (voltage) float64 -29.97 -29.95 -29.93 ... 0.225 0.25
    probe                 (temperature) object 'flat_flush_as' 'flat_flush_T10'
    recession             (temperature) float64 0.0 0.0
    recession_desc        (temperature) object 'flush' 'flush'
    recessed_fl           (temperature) bool False False
    theta_p               (temperature) float64 0.0 0.0
    theta_p_desc          (temperature) object 'flat' 'flat'
    angled_fl             (temperature) bool False False
    theta_p_rads          (temperature) float64 0.0 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.1745 0.5236
Data variables:
    str_iv_chi2           (temperature, theta) float64 4.549e+03 ... 1.39e+03
    str_iv_reduced_chi2   (temperature, theta) float64 4.08 5.962 ... 1.248
    str_iv_I_sat          (temperature, theta) float64 -7.115 -10.62 ... -64.3
    str_iv_d_I_sat        (temperature, theta) float64 0.04334 ... 0.2257
    str_iv_a              (temperature, theta) float64 0.006474 ... 0.008666
    str_iv_d_a            (temperature, theta) float64 0.00111 ... 0.0003635
    str_iv_T_e            (temperature, theta) float64 1.538 1.429 ... 0.4799
    str_iv_d_T_e          (temperature, theta) float64 0.01072 ... 0.02535
    str_iv_V_f            (temperature, theta) float64 -4.628 -4.493 ... -1.751
    str_iv_d_V_f          (temperature, theta) float64 0.009595 ... 0.02926
    str_iv_run_dir        (temperature, theta) object 'marconi/spice2/sheath_exp_fwp/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/se_tempscan/flat_flush_T10/alpha_yz_-30.0'
    str_iv_voltage_min    (temperature, theta) float64 -29.97 -29.97 ... -29.97
    str_iv_voltage_max    (temperature, theta) float64 -2.025 -2.025 ... -2.025
    norm_iv_chi2          (temperature, theta) float64 695.7 780.1 ... 2.332e+05
    norm_iv_reduced_chi2  (temperature, theta) float64 0.6861 0.7641 ... 193.5
    norm_iv_I_sat         (temperature, theta) float64 -6.516 -9.839 ... -47.29
    norm_iv_d_I_sat       (temperature, theta) float64 0.01378 0.0216 ... 2.701
    norm_iv_a             (temperature, theta) float64 0.01583 ... 0.05724
    norm_iv_d_a           (temperature, theta) float64 0.0003211 ... 0.009949
    norm_iv_voltage_min   (temperature, theta) float64 -25.38 -25.57 ... -30.2
    norm_iv_voltage_max   (temperature, theta) float64 -0.0008451 ... -0.025
    ion_chi2              (temperature, theta) float64 513.2 556.5 ... 1.312e+03
    ion_reduced_chi2      (temperature, theta) float64 0.601 0.6463 ... 1.255
    ion_I_sat             (temperature, theta) float64 6.573 9.791 ... 63.15
    ion_d_I_sat           (temperature, theta) float64 0.01636 ... 0.2543
    ion_a                 (temperature, theta) float64 0.01465 ... 0.01606
    ion_d_a               (temperature, theta) float64 0.0003592 ... 0.0005284
    ion_voltage_min       (temperature, theta) float64 2.829 2.841 ... 2.828
    ion_voltage_max       (temperature, theta) float64 11.31 11.37 ... 12.88
    current               (temperature, theta, voltage) float64 -7.778 ... -0.0
    current_e             (temperature, theta, voltage) float64 0.0 0.0 ... 0.0
    current_i             (temperature, theta, voltage) float64 7.778 ... 0.0
    d_current             (temperature, theta, voltage) float64 0.2109 ... 0.0
    d_current_e           (temperature, theta, voltage) float64 0.0 0.0 ... 0.0
    d_current_i           (temperature, theta, voltage) float64 0.2109 ... 0.0
    time                  (temperature, theta, voltage) float64 0.0 ... 5.274

In [63]:
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True, figsize=[10,10])
voltage_slice = slice(-15,0)

for j, tempscan_ds in enumerate([tempscan_1_ds, tempscan_2_ds]):
    for i, temp in enumerate(tempscan_ds.temperature.values):
        ax = axes[j,i]
        
        tempscan_ds.sel(temperature=temp, voltage=voltage_slice)['current_e'].plot.line(hue='theta', ax=ax)
fig.tight_layout()

In [65]:
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True, figsize=[10,10])
voltage_slice = slice(-15,-7)

for j, tempscan_ds in enumerate([tempscan_1_ds, tempscan_2_ds]):
    for i, temp in enumerate(tempscan_ds.temperature.values):
        ax = axes[j,i]
        for theta in tempscan_ds.theta.values:
            tempscan_plot_ds = tempscan_ds.sel(temperature=temp, voltage=voltage_slice, theta=theta)
            ax.plot(mod_pow(tempscan_plot_ds.voltage.values, v_f=4), tempscan_plot_ds.current_i.values)
#         tempscan_ds.sel(temperature=temp)['ion_a'].plot.line(hue='theta', ax=ax, ls='none', marker='.')
fig.tight_layout()

In [61]:
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True, figsize=[10,10])
voltage_slice = slice(-30,-7)

for j, tempscan_ds in enumerate([tempscan_1_ds, tempscan_2_ds]):
    for i, temp in enumerate(tempscan_ds.temperature.values):
        ax = axes[j,i]
        for theta in tempscan_ds.theta.values:
            tempscan_plot_ds = tempscan_ds.sel(temperature=temp, voltage=voltage_slice, theta=theta)
            ax.plot(mod_pow(tempscan_plot_ds.voltage.values, v_f=4), -tempscan_plot_ds.current_e.values)
#         tempscan_ds.sel(temperature=temp)['ion_a'].plot.line(hue='theta', ax=ax, ls='none', marker='.')
fig.tight_layout()

In [58]:
# measure the log plot
fig, ax = plt.subplots()

## Look at ion current slope form

In [55]:
example_ds = combined_ds.sel(theta=8, probe='flat_flush')
example_ds = example_ds.drop('str_iv_run_dir').where(example_ds['current'] < 0, drop=True)
example_ds

<xarray.Dataset>
Dimensions:               (voltage: 1104)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 -29.92 ... -2.425 -2.4
    theta                 float64 8.0
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            float64 0.1396
    probe                 <U10 'flat_flush'
Data variables:
    str_iv_chi2           (voltage) float64 982.8 982.8 982.8 ... 982.8 982.8
    str_iv_reduced_chi2   (voltage) float64 0.8814 0.8814 ... 0.8814 0.8814
    str_iv_I_sat          (voltage) float64 -57.63 -57.63 ... -57.63 -57.63
    str_iv_d_I_sat        (voltage) float64 0.07858 0.07858 ... 0.07858 0.07858
    str_iv_a              (voltage) float64 0.007449 0.007449 ... 0.007449
    str_iv_d_a            (voltage) float64 0.000182 0.000182 ... 0.000182
    str_iv_T_e            (voltage) float64 1.171 1.171 1.171 ... 1.171 1.171
    str_iv_d_T_e          (voltage) float64 0.006288 0.006288 ... 0.006288
    str_iv_V_f            (voltage) float64 -2.414 -2.414 ... -2.414 -2.414
    str_iv_d_V_f          (voltage) float64 0.003094 0.003094 ... 0.003094
    str_iv_voltage_min    (voltage) float64 -29.98 -29.98 ... -29.98 -29.98
    str_iv_voltage_max    (voltage) float64 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (voltage) float64 1.913e+03 1.913e+03 ... 1.913e+03
    norm_iv_reduced_chi2  (voltage) float64 1.736 1.736 1.736 ... 1.736 1.736
    norm_iv_I_sat         (voltage) float64 -56.16 -56.16 ... -56.16 -56.16
    norm_iv_d_I_sat       (voltage) float64 0.08549 0.08549 ... 0.08549 0.08549
    norm_iv_a             (voltage) float64 0.009707 0.009707 ... 0.009707
    norm_iv_d_a           (voltage) float64 0.0002087 0.0002087 ... 0.0002087
    norm_iv_voltage_min   (voltage) float64 -27.58 -27.58 ... -27.58 -27.58
    norm_iv_voltage_max   (voltage) float64 -0.003501 -0.003501 ... -0.003501
    ion_chi2              (voltage) float64 664.3 664.3 664.3 ... 664.3 664.3
    ion_reduced_chi2      (voltage) float64 0.7707 0.7707 ... 0.7707 0.7707
    ion_I_sat             (voltage) float64 57.87 57.87 57.87 ... 57.87 57.87
    ion_d_I_sat           (voltage) float64 0.08557 0.08557 ... 0.08557 0.08557
    ion_a                 (voltage) float64 0.006133 0.006133 ... 0.006133
    ion_d_a               (voltage) float64 0.0001831 0.0001831 ... 0.0001831
    ion_voltage_min       (voltage) float64 3.835 3.835 3.835 ... 3.835 3.835
    ion_voltage_max       (voltage) float64 12.03 12.03 12.03 ... 12.03 12.03
    current               (voltage) float64 -62.71 -62.52 ... -0.3439 -0.1858
    current_e             (voltage) float64 0.0 0.0 0.0 ... -49.04 -51.58 -52.17
    current_i             (voltage) float64 62.71 62.52 63.78 ... 51.93 52.36
    d_current             (voltage) float64 0.8428 0.8524 0.9034 ... 1.007 1.048
    d_current_e           (voltage) float64 0.0 0.0 0.0 ... 0.7554 0.7802 0.7512
    d_current_i           (voltage) float64 0.8428 0.8524 ... 0.6196 0.6698
    time                  (voltage) float64 0.0 0.01613 0.03226 ... 17.78 17.79
    recession             (voltage) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [52]:
fig, ax = plt.subplots(2, sharex=True)
example_ds['current'].plot.line(x='voltage', ax=ax[0])
example_ds['current_i'].plot.line(x='voltage', ax=ax[1])

In [77]:
fig, ax = plt.subplots(2)
log_ds = example_ds.sel(voltage=slice(-27.5,-5))
(example_ds['current_i'] / example_ds['ion_I_sat']).plot.line(x='voltage', ax=ax[0])
(log_ds['current_i'] / log_ds['ion_I_sat']).plot.line(x='voltage', ax=ax[0])

y = np.log((log_ds['current_i'] / log_ds['ion_I_sat']) - 1)
x = np.log(np.abs(log_ds['voltage']))

x = x[~np.isnan(y)]
y = y[~np.isnan(y)]

str_fitter = fts.StraightLineFitter()
fit_data = str_fitter.fit(x, y)
ax[1].plot(x, y, '+')
ax[1].plot(*fit_data.get_fit_plottables(), label=fit_data.get_param_str())

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/computation.py:604: RuntimeWarning: invalid value encountered in log
  result_data = func(*input_data)


In [76]:
print(np.exp(fit_data.get_param('y_0')))
print(log_ds.ion_a.values[0])

0.00662188695198328
0.006132512526980963
